## 2. Structure (Part 1)
In this notebook we will take the data we prepared and that is now found in <code>cleaned_logs/</code>. At the end of this notebook, we will end up with several files that are structured with meaningful data analysis in mind:

- <code>ping.csv</code>
- <code>routing.csv</code>
- <code>mesh.csv</code>
- <code>wifi.csv</code>

In the next notebook we will further add granularity to support various analysis purposes.


### 2.1 Master Files
#### 2.1.1 Wifi
The `wifi` file only consist of `no_of_connections` and therefore the final file will consist of three columns: 

- `timestamp`
- `node_ip`
- `no_of_connections`

<i><font style='color:red'>NOTE: it's not clear if the files share the timestamp, so merge files later keeping this in mind.</font></i>

In [10]:
def _wifi_clients(node_ip):

    # open and read the file for the given node
    f = open('cleaned_logs/' + node_ip + '_wifi.log', 'r')
    out = f.readlines()

    final = []

    # iterate through each record to clean it up
    for i in out:
        
        # remove extra markings
        i = i.strip()

        # the records start with date so store that first
        if i.startswith('###'):
            date = ' '.join(i.split()[2:])
            
        # then take the actual values and combine both into single record
        elif 'Number' in i:
            data = i.split(':')[-1]
            final.append([node_ip, date, data])
            
    # return the cleaned records
    return final

def create_connections_csv():

    import pandas as pd
    import os

    # get a list of unique node IPs
    nodes = []
    for i in os.popen('ls cleaned_logs').readlines():
        i = i.strip()
        nodes.append(i.split('_')[0])
        nodes = list(set(nodes))

    # go through files for each and clean records
    out = []
    for node in nodes:
        data = _wifi_clients(node)
        out += data

    # create a dataframe from the records
    wifi_df = pd.DataFrame(out)
    wifi_df.columns = ['node_ip', 'timestamp', 'no_of_connections']
    wifi_df.no_of_connections = wifi_df.no_of_connections.astype(int)
    
    # convert the timestamp string into datetime column
    wifi_df.timestamp = pd.DatetimeIndex(wifi_df.timestamp)

    # export to CSV in pwd
    wifi_df.to_csv('wifi.csv', index=None)

In [11]:
create_connections_csv()

#### 2.1.2 Ping
The `ping` file only consist of several metrics that are returned from a standard ping command. Like all the other files, it also consist of `node_ip` and `timestamp`. Comprehensive list of the columns in ping file: 

- `min`
- `avg`
- `max`
- `std`
- `node_ip`
- `no_of_connections`

In [16]:
def create_ping_csv():
    
    import pandas as pd
    import os

    # get a list of unique node IPs
    nodes = []
    for i in os.popen('ls cleaned_logs').readlines():
        i = i.strip()
        nodes.append(i.split('_')[0])
        nodes = list(set(nodes))

    # go through files for each and clean records
    out = []
    for node in nodes:
        data = _ping_stats(node)
        out += data

    
    # create a dataframe from the records
    ping_df = pd.DataFrame(out)
    ping_df.columns = ['node_ip', 'timestamp', 'min', 'avg', 'max', 'mdev']
    ping_df[['min', 'avg', 'max', 'mdev']] = ping_df[['min', 'avg', 'max', 'mdev']].astype(float)
    
    # convert the timestamp string into datetime column
    ping_df.timestamp = pd.DatetimeIndex(ping_df.timestamp)

    # export to CSV in pwd
    ping_df.to_csv('ping.csv', index=None)


# helper function for create_ping_csv()
def _ping_stats(node_ip):

    # open and read the file for the given node
    f = open('cleaned_logs/' + node_ip + '_ping.log', 'r')
    out = f.readlines()

    final = []
    
    # iterate through each record to clean it up
    for i in out:
        
        # remove extra markings
        i = i.strip()

        # the records start with date so store that first
        if i.startswith('###'):
            date = ' '.join(i.split()[2:])
        
        # then take the actual values and combine both into single record
        elif 'rtt' in i:
            data = i.split('=')[-1].split(' ')[1]
            data = data.split('/')
            final.append([node_ip, date] + data)
    
    # return the cleaned records
    return final

#### 2.1.3 Mesh
The `mesh` file consist of many metrics, all associated with the Quality-of-Service (QOS) aspect of the mesh network:

- `node_ip`
- `timestamp`
- `node_mac`
- `signal_dbm`
- `noise_dbm`
- `ago_ms`
- `rx_mbps`
- `rx_MCS`
- `rx_MHz`
- `rx_pkts`
- `tx_mbps`
- `tx_MCS`
- `tx_MHz`
- `tx_pkts`


In [20]:
def create_mesh_csv():
    
    import os
    import numpy as np
    import pandas as pd

    # get the node IPs
    nodes = list(set([i.split('_')[0] for i in os.popen('ls cleaned_logs/').readlines()]))

    # clean up the records for each node
    out = []
    for node in nodes:
        data = _mesh(node)
        out += data
    
    # create a dataframe from the cleaned records
    mesh_df = pd.DataFrame(out)
    mesh_df.columns = ['node_ip',
                       'timestamp',
                       'node_mac',
                       'signal_dbm',
                       'noise_dbm',
                       'ago_ms',
                       'rx_mbps',
                       'rx_MCS',
                       'rx_MHz',
                       'rx_pkts',
                       'tx_mbps',
                       'tx_MCS',
                       'tx_MHz',
                       'tx_pkts']

    # clean up various mess from the original records
    mesh_df = mesh_df.replace('NA', np.nan)
    mesh_df = mesh_df.replace('unknown', np.nan)
    mesh_df = mesh_df.replace('ms', np.nan)
    mesh_df = mesh_df.replace('dBm', np.nan)
    mesh_df.rx_MHz = mesh_df.rx_MHz.str.replace('MHz', '')
    mesh_df.tx_MHz = mesh_df.tx_MHz.str.replace('MHz', '')
    mesh_df.rx_MCS = mesh_df.rx_MCS.str.replace(',', '')
    mesh_df.tx_MCS = mesh_df.tx_MCS.str.replace(',', '')

    # convert to float what readily converts
    for col in mesh_df.columns:
        try:
            mesh_df[col] = mesh_df[col].astype(float)
        except:
            pass

    # drop bad records
    mesh_df = mesh_df[~mesh_df.timestamp.str.contains('###')]

    # convert timestamp string to datetime column
    mesh_df.timestamp = pd.DatetimeIndex(mesh_df.timestamp)

    # export to csv
    mesh_df.to_csv('mesh.csv', index=None)


def _mesh(node_ip):

    f = open('cleaned_logs/' + node_ip + '_mesh.log', 'r')
    out = f.readlines()

    final = []

    for i in out:

        i = i.strip()

        if i.startswith('###'):
            date = ' '.join(i.split()[2:])
        
        elif 'ms ago' in i:
            header = i.split()
            header = [header[0], header[1], header[4], header[8]]
            
        elif 'RX:' in i:
            rx = i.split()
            try:
                rx = [rx[1], rx[4], rx[5], rx[6]]
            except IndexError:
                rx = ['NA', 'NA', 'NA', 'NA']
            
        elif 'TX:' in i:
            tx = i.split()
            try:
                tx = [tx[1], tx[4], tx[5], tx[6]]
            except IndexError:
                tx = ['NA', 'NA', 'NA', 'NA']
            
            final.append([node_ip, date] + header + rx + tx)
            
    return final